In [ ]:
!pip install -U duckduckgo_search
# Doc: https://pypi.org/project/duckduckgo-search/

In [ ]:
!pip install newspaper3k

In [ ]:
!pip install mechanize

In [5]:
from duckduckgo_search import DDGS
from newspaper import Article
import json

In [3]:
from duckduckgo_search import ddg

In [ ]:
with DDGS() as ddgs:
    keywords = '"huracán otis"'
    ddgs_news_gen = ddgs.text(
      keywords,
      region="mx-es",
      safesearch="off",
      backend='lite',
      timelimit='m',
      max_results=500
    )
    
urls = []  # Inicializamos una lista vacía para almacenar las URLs
for r in ddgs_news_gen:
    urls.append(r['href'])

In [21]:
results = ddg('"huracán otis"', max_results=100)
urls = [result['href'] for result in results]

In [22]:
urls

['https://www.bbc.com/mundo/articles/c807nr77p4xo',
 'https://es.wikipedia.org/wiki/Huracán_Otis',
 'https://www.bbc.com/mundo/articles/cjl03ep04yyo',
 'https://ciencia.nasa.gov/ciencias-terrestres/el-huracan-otis/',
 'https://www.cnn.com/videos/spanish/2023/10/25/hurcan-otis-acapulco-orix.cnn',
 'https://www.cnn.com/videos/spanish/2023/10/25/otis-huracan-acapulco-guerrero-perspectivas-mexico-tv.cnn',
 'https://cnnespanol.cnn.com/2023/10/24/tormenta-tropical-otis-fuerza-huracan-sur-mexico-trax/',
 'https://www.bloomberg.com/news/articles/2023-10-25/huracan-otis-azota-mexico-tras-tocar-tierra-cerca-de-acapulco',
 'https://edition.cnn.com/videos/spanish/2023/10/31/victimas-huracan-otis-entierro-cuerpos-acapulco-conclusiones-tv.cnn',
 'https://cnnespanol.cnn.com/2023/10/27/huracan-otis-danos-noticias-acapulco-mexico-orix-2/',
 'https://www.latimes.com/espanol/mexico/articulo/2023-10-29/victimas-por-paso-del-huracan-otis-se-elevan-a-43-en-el-sur-de-mexico-y-los-desaparecidos-suben-a-36',
 

In [9]:
import mechanize
import re
import time

def extract_urls(query, num_urls=100):
    # Configurar el navegador
    br = mechanize.Browser()
    br.set_handle_robots(False)  # Ignorar el archivo robots.txt
    br.addheaders = [('User-agent', 'Mozilla/5.0')]  # Definir el User-agent

    # Abrir DuckDuckGo con parámetros para español (México)
    base_url = "https://html.duckduckgo.com/html/?kl=mx-es&t=h_&iar=news&df=w&ia=news"
    br.open(base_url)

    # Seleccionar el formulario de búsqueda y enviar la consulta
    br.select_form(nr=0)
    br["q"] = query
    response = br.submit()

    urls = set()

    # Regex para extraer URLs
    link_pattern = re.compile(r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\\(\\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+')

    while len(urls) < num_urls:
        # Extraer URLs de la página actual
        for link in br.links():
            match = link_pattern.match(link.url)
            if match and link.url not in urls:  # Verifica si la URL no está ya en el conjunto
                urls.add(link.url)
                print(link.url)  # Imprime la URL recién agregada
                if len(urls) == num_urls:
                    return list(urls)
        
        # Intentar enviar el formulario "Next" para la próxima página de resultados
        try:
            br.select_form(nr=1)
            br.submit()
        except (mechanize.FormNotFoundError, IndexError):
            break

        time.sleep(2)

    return list(urls)

# Prueba
#urls = extract_urls('"El huracán Otis"')

#urls = extract_urls('huracán otis acapulco"')

urls = extract_urls('apoyo damnificados acopio otis acapulco"')




https://elpais.com/mexico/2023-10-31/centros-de-acopio-para-acapulco-como-ayudar-a-los-damnificados-por-otis.html
https://www.milenio.com/estados/marina-traslada-despensas-y-agua-a-damnificados-por-otis-en-acapulco
https://www.dondeir.com/noticias/centros-de-acopio-para-apoyar-a-damnificados-por-huracan-otis-en-acapulco/2023/10/
https://www.milenio.com/politica/comunidad/huracan-otis-cruz-roja-nuevo-leon-envia-apoyo-guerrero
https://www.eluniversal.com.mx/nacion/centro-de-acopio-cruz-roja-que-se-puede-donar-para-los-damnificados-de-acapulco/
https://heraldodemexico.com.mx/espectaculos/2023/11/2/huracan-otis-estos-famosos-se-han-unido-para-apoyar-damnificados-de-acapulco-551761.html
https://www.de10.com.mx/liga-mx/fanaticada-universitaria-despliega-manta-en-apoyo-a-acapulco-como-no-te-vamos-a-apoyar/
https://politica.expansion.mx/cdmx/2023/10/31/politicos-de-la-cdmx-anuncian-apoyos-para-damnificados-en-acapulco
https://www.eldiariodechihuahua.mx/nacional/extienden-cadena-de-apoyo-a-damn

In [ ]:
for url in results:
    print(url)

In [ ]:
urls

In [3]:
def extract_content_from_url(url):
    try:
        article = Article(url)
        article.download()
        article.parse()
        return {
            "title": article.title,
            "content": article.text,
            "url": url,
            "date": article.publish_date.strftime('%Y-%m-%d') if article.publish_date else None,
            "image_url": article.top_image
        }
    except Exception as e:
        print(f"Error al extraer contenido de {url}. Error: {e}")
        return None

In [10]:
import os

# Definimos la ruta del archivo
archivo_path = './02_news/noticias_huracan_otis.jsonl'

# Diccionario para almacenar las noticias, usando la URL como llave
noticias_dict = {}

# Si el archivo ya existe, lo leemos para cargar las noticias existentes
if os.path.exists(archivo_path):
    with open(archivo_path, 'r', encoding='utf-8') as file:
        for line in file:
            noticia = json.loads(line.strip())
            noticias_dict[noticia['url']] = noticia

# Procesamos las URLs nuevas
for r in urls:
    print(r)
    if r not in noticias_dict:  # Verificamos que la URL no exista ya en el diccionario
        content = extract_content_from_url(r)
        if content:
            noticias_dict[content['url']] = content  # Agregamos la noticia al diccionario

# Guardamos las noticias (tanto las antiguas como las nuevas) en el archivo
with open(archivo_path, 'w', encoding='utf-8') as file:
    for noticia in noticias_dict.values():
        file.write(json.dumps(noticia, ensure_ascii=False) + '\n')

print("¡Proceso completado!")


https://mimorelia.com/noticias/morelia/contin%C3%BAa-abierto-centro-de-acopio-del-imss-en-apoyo-a-damnificados-de-acapulco
https://www.jornada.com.mx/noticia/2023/10/31/estados/mantendran-de-forma-indefinida-centro-de-acopio-en-toluca-761
https://www.milenio.com/politica/comunidad/acolman-envia-cargamento-viveres-damnificados-otis
Error al extraer contenido de https://www.milenio.com/politica/comunidad/acolman-envia-cargamento-viveres-damnificados-otis. Error: Article `download()` failed with HTTPSConnectionPool(host='www.milenio.com', port=443): Read timed out. (read timeout=7) on URL https://www.milenio.com/politica/comunidad/acolman-envia-cargamento-viveres-damnificados-otis
https://heraldodemexico.com.mx/espectaculos/2023/11/2/huracan-otis-estos-famosos-se-han-unido-para-apoyar-damnificados-de-acapulco-551761.html
https://elpais.com/mexico/2023-10-31/centros-de-acopio-para-acapulco-como-ayudar-a-los-damnificados-por-otis.html
https://mvsnoticias.com/nuevo-leon/2023/10/30/estos-son-